# Training gemma

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\Users\VICTUS\Documents\developer\UIT_year3_sem1\cs221-law_retrieval_system\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa"
)

c:\Users\VICTUS\Documents\developer\UIT_year3_sem1\cs221-law_retrieval_system\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VICTUS\.cache\huggingface\hub\models--google--gemma-2-2b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]Xet Storage 